
# ⚽ Live Match Analysis — Quickstart Notebook

This notebook demonstrates a **live, end‑to‑end flow** using your router:
1) Fetch **live matches** right now  
2) Auto-pick the **first** live match  
3) Run **analysis** features you added (insights, recent form, win probabilities, H2H)  
4) Pull an **enriched timeline** with lightweight event tags  
5) (Optional) Fetch **highlight videos**

> **Prereqs**
> - Your repo environment is active (the same one where you got `Router ready ✅`).  
> - `ALLSPORTS_API_KEY` is set as an environment variable.  
> - Notebook kernel can import your code (we add your project root to `sys.path` below).



## 1) Environment check & imports
This cell:
- Finds your project root (the folder that contains `backend/app`)
- Adds it to `sys.path`
- Verifies your `ALLSPORTS_API_KEY`
- Imports and instantiates the `RouterCollector`


In [1]:

import os, sys, json
from pathlib import Path

def find_project_root(max_hops: int = 8) -> Path | None:
    p = Path.cwd()
    for _ in range(max_hops):
        if (p / "backend" / "app").exists():
            return p
        p = p.parent
    return None

PROJECT_ROOT = find_project_root()
if not PROJECT_ROOT:
    raise RuntimeError("Could not locate project root (folder containing backend/app). Start the notebook from inside your repo.")

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Using project root:", PROJECT_ROOT)

# Check API key
api_ok = bool(os.environ.get("ALLSPORTS_API_KEY"))
print("ALLSPORTS_API_KEY set:", api_ok)

# Import router
from backend.app.routers.router_collector import RouterCollector

router = RouterCollector()
print("Router ready \u2705")


Using project root: /Users/vishwaperera/Desktop/Projects/Sports-Analysis/sports-ai
ALLSPORTS_API_KEY set: True
Router ready ✅



## 2) Small helpers
- `extract_events_from_payload` – normalizes provider payloads into a simple list of event dicts
- `as_df` – convenience for quick DataFrame rendering


In [2]:

from typing import Any, Dict, List, Iterable

def extract_events_from_payload(data: Dict[str, Any]) -> List[Dict[str, Any]]:
    """Accepts AllSports/TSDB shapes and returns a flat list of event dicts."""
    if not isinstance(data, dict):
        return []
    for key in ("result", "events", "results", "fixtures"):
        v = data.get(key)
        if isinstance(v, list):
            return v
    return []

def as_df(rows: Iterable[dict]):
    try:
        import pandas as pd
        return pd.DataFrame(list(rows))
    except Exception:
        # pandas not installed — return raw list
        return list(rows)



## 3) Get live matches
We call `events.live` (AllSports primary). The helper extracts the match list from the provider body.


In [5]:

live = router.handle({"intent": "events.live", "args": {}})
live_ok = bool(live.get("ok"))
live_list = extract_events_from_payload(live.get("data") or {})
print("OK:", live_ok, "| live matches:", len(live_list))

# Show the first ~15 live matches in a compact table
core = []
for ev in live_list[:50]:
    core.append({
        "event_key": ev.get("event_key") or ev.get("idEvent") or ev.get("id") or ev.get("fixture_id"),
        "date": ev.get("event_date") or ev.get("dateEvent"),
        "time": ev.get("event_time") or ev.get("strTime"),
        "league": ev.get("league_name") or ev.get("strLeague"),
        "home": ev.get("event_home_team") or ev.get("strHomeTeam"),
        "away": ev.get("event_away_team") or ev.get("strAwayTeam"),
        "status": ev.get("event_status") or ev.get("status")
    })

df_live = as_df(core)
df_live


OK: True | live matches: 32


,event_key,date,time,league,home,away,status
0,1656877,2025-09-03,17:00,DBU Pokalen - 2nd Round,B 1909,Hobro,90+
1,1672853,2025-09-03,18:30,Club Friendlies,Grossaspach,Stuttgart,12
2,1672852,2025-09-03,17:30,Club Friendlies,Andorra,Inter Escaldes,60
3,1672337,2025-09-03,18:00,Cup - 1st Round,Denizli IY Gureller,Nazillispor,Half Time
4,1672335,2025-09-03,18:00,Cup - 1st Round,Karaman,Suvermez,Half Time
5,1672334,2025-09-03,18:00,Cup - 1st Round,Fethiyespor,Soke 1970,Half Time
6,1672188,2025-09-03,17:30,QSL Cup - Group Stage,Al-Shahaniya,Al Sailiya,57
7,1672187,2025-09-03,17:30,QSL Cup - Group Stage,Al Khor,Al Bidda,62
8,1667547,2025-09-03,18:00,Azadegan League,Sanat Naft,Saipa,Half Time
9,1667546,2025-09-03,18:00,Azadegan League,Pars Janoobi Jam,Nassaji Mazandaran,39



## 4) Pick the first live match
We'll grab the first item from the live list and resolve a unified `eventId` from common keys.


In [6]:

if not live_list:
    raise RuntimeError("No live matches at the moment. Re-run later when there are live games.")

chosen = live_list[16]
def normalize_event_id(ev: Dict[str, Any]) -> str | None:
    for k in ("event_key","idEvent","event_id","match_id","fixture_id","id","eventId"):
        v = ev.get(k)
        if v is not None and str(v).strip() != "":
            return str(v)
    return None

event_id = normalize_event_id(chosen)
if not event_id:
    raise RuntimeError("Could not resolve eventId from the chosen live match.")

home = chosen.get("event_home_team") or chosen.get("strHomeTeam")
away = chosen.get("event_away_team") or chosen.get("strAwayTeam")
league = chosen.get("league_name") or chosen.get("strLeague")
print(f"Chosen live match: [{league}] {home} vs {away} — eventId={event_id}")
chosen


Chosen live match: [II Liga] Swit Szczecin vs Kleczew — eventId=1613510


{'event_key': 1613510,
 'event_date': '2025-09-03',
 'event_time': '17:00',
 'event_home_team': 'Swit Szczecin',
 'home_team_key': 5791,
 'event_away_team': 'Kleczew',
 'away_team_key': 5852,
 'event_halftime_result': '1 - 1',
 'event_final_result': '3 - 1',
 'event_ft_result': '',
 'event_penalty_result': '',
 'event_status': '90+',
 'country_name': 'Poland',
 'league_name': 'II Liga',
 'league_key': 261,
 'league_round': 'Round 6',
 'league_season': '2025/2026',
 'event_live': '1',
 'event_stadium': 'Stadion Miejski Skolwin',
 'event_referee': '',
 'home_team_logo': 'https://apiv2.allsportsapi.com/logo/5791_wit-szczecin.jpg',
 'away_team_logo': 'https://apiv2.allsportsapi.com/logo/5852_soko-kleczew.jpg',
 'event_country_key': 91,
 'league_logo': 'https://apiv2.allsportsapi.com/logo/logo_leagues/261_ii-liga.png',
 'country_logo': 'https://apiv2.allsportsapi.com/logo/logo_country/91_poland.png',
 'event_home_formation': '',
 'event_away_formation': '',
 'fk_stage_key': 1428,
 'stage_na


## 5) Match insights
High-level insights about this specific fixture (your `analysis.match_insights` intent).


In [7]:

insights = router.handle({"intent": "analysis.match_insights", "args": {"eventId": str(event_id)}})
print("OK:", insights.get("ok"))
# Display the raw structure so you can see what's available from your agent
insights


OK: True


{'ok': True,
 'intent': 'analysis.match_insights',
 'args_resolved': {'eventId': '1613510'},
 'data': {'winprob': {'eventId': '1613510',
   'method': 'form_logistic',
   'probs': {'home': 0.3927199430614866,
    'draw': 0.2857481622070403,
    'away': 0.32153189473147303},
   'inputs': {'home_metrics': {'games': 0,
     'wins': 0,
     'draws': 0,
     'losses': 0,
     'gf': 0,
     'ga': 0,
     'gd': 0,
     'ppg': 0.0,
     'gd_per_game': 0.0,
     'last_results': [],
     'win_streak': 0,
     'unbeaten': 0,
     'streak_bonus': 0.0},
    'away_metrics': {'games': 0,
     'wins': 0,
     'draws': 0,
     'losses': 0,
     'gf': 0,
     'ga': 0,
     'gd': 0,
     'ppg': 0.0,
     'gd_per_game': 0.0,
     'last_results': [],
     'win_streak': 0,
     'unbeaten': 0,
     'streak_bonus': 0.0}}},
  'form': {'eventId': '1613510',
   'home_team': {'id': '5791',
    'name': 'Swit Szczecin',
    'summary': '0-0-0 last 0'},
   'away_team': {'id': '5852', 'name': 'Kleczew', 'summary': '0-0


## 6) Win probabilities
We request `analysis.winprob` and render a simple bar chart of Home / Draw / Away.


In [8]:

winprob = router.handle({"intent": "analysis.winprob", "args": {"eventId": str(event_id)}})
print("OK:", winprob.get("ok"))

data = winprob.get("data") or {}
# Try common shapes: {'home':..,'draw':..,'away':..} OR nested
if isinstance(data, dict) and all(k in data for k in ("home","draw","away")):
    probs = data
else:
    # look inside 'probabilities' or 'result' shapes
    probs = None
    for k in ("probabilities","result","data"):
        v = data.get(k) if isinstance(data, dict) else None
        if isinstance(v, dict) and all(x in v for x in ("home","draw","away")):
            probs = v
            break

if not probs:
    print("Could not find home/draw/away fields. Dumping payload for inspection:")
    display(winprob)
else:
    print("Win probabilities:", probs)
    # --- Bar chart (matplotlib, single plot, no custom colors) ---
    import matplotlib.pyplot as plt
    labels = ["Home","Draw","Away"]
    values = [float(probs.get("home",0))*100, float(probs.get("draw",0))*100, float(probs.get("away",0))*100]

    plt.figure(figsize=(5,3.5))
    plt.bar(labels, values)
    plt.ylabel("%")
    plt.title("Win Probabilities")
    plt.ylim(0, 100)
    plt.show()


OK: True
Could not find home/draw/away fields. Dumping payload for inspection:


{'ok': True,
 'intent': 'analysis.winprob',
 'args_resolved': {'eventId': '1613510'},
 'data': {'eventId': '1613510',
  'method': 'form_logistic',
  'probs': {'home': 0.3927199430614866,
   'draw': 0.2857481622070403,
   'away': 0.32153189473147303},
  'inputs': {'home_metrics': {'games': 0,
    'wins': 0,
    'draws': 0,
    'losses': 0,
    'gf': 0,
    'ga': 0,
    'gd': 0,
    'ppg': 0.0,
    'gd_per_game': 0.0,
    'last_results': [],
    'win_streak': 0,
    'unbeaten': 0,
    'streak_bonus': 0.0},
   'away_metrics': {'games': 0,
    'wins': 0,
    'draws': 0,
    'losses': 0,
    'gf': 0,
    'ga': 0,
    'gd': 0,
    'ppg': 0.0,
    'gd_per_game': 0.0,
    'last_results': [],
    'win_streak': 0,
    'unbeaten': 0,
    'streak_bonus': 0.0}}},
 'error': None,
 'meta': {'source': {'primary': 'analysis', 'fallback': 'allsports'},
  'trace': [{'step': 'sports.event.get',
    'ok': True,
    'raw_meta': {'trace': [{'step': 'allsports_call',
       'met': 'Fixtures',
       'status':


## 7) Recent form (lookback configurable)
We request `analysis.form` with a lookback window (default 5). This returns recent performance context for this fixture.


In [ ]:

form = router.handle({"intent": "analysis.form", "args": {"eventId": str(event_id), "lookback": 5}})
print("OK:", form.get("ok"))
form



## 8) Head‑to‑Head (H2H)
We request `analysis.h2h` for the same `eventId` (you can also pass `teamA`/`teamB` names instead).


In [ ]:

h2h = router.handle({"intent": "analysis.h2h", "args": {"eventId": str(event_id), "lookback": 5}})
print("OK:", h2h.get("ok"))
h2h



## 9) Enriched timeline with lightweight tags
We fetch the raw event (`event.get`) and ask for tag augmentation. The agent will synthesize a `timeline` when missing and add simple tags (e.g., GOAL, YELLOW_CARD).


In [ ]:

evt = router.handle({"intent": "event.get", "args": {"eventId": str(event_id), "augment_tags": True}})
print("OK:", evt.get("ok"))

# Try to locate the event dict and a 'timeline' list within the provider body
def find_event_and_timeline(payload: dict):
    data = payload.get("data") or {}
    ev = None
    # common shapes
    if isinstance(data, dict) and isinstance(data.get("result"), list) and data.get("result"):
        ev = data["result"][0]
    elif isinstance(data, dict) and data.get("event") and isinstance(data.get("event"), dict):
        ev = data["event"]
    elif isinstance(data, dict) and (data.get("events") or data.get("fixtures")):
        col = data.get("events") or data.get("fixtures")
        if isinstance(col, list) and col:
            ev = col[0]
    elif isinstance(data, dict) and data:
        # last resort: first dict-like nested value that looks like an event
        for v in data.values():
            if isinstance(v, dict) and (v.get("event_key") or v.get("idEvent")):
                ev = v
                break

    timeline = None
    if isinstance(ev, dict):
        for k in ("timeline","timeline_items","events","event_timeline"):
            if isinstance(ev.get(k), list):
                timeline = ev.get(k)
                break
    return ev, timeline

ev_obj, timeline = find_event_and_timeline(evt)
print("Has event object:", isinstance(ev_obj, dict), "| timeline items:", (len(timeline) if isinstance(timeline, list) else 0))

# Show a compact view of first few timeline items (minute, description, predicted_tags)
rows = []
if isinstance(timeline, list):
    for it in timeline[:12]:
        rows.append({
            "minute": it.get("minute") or it.get("time") or it.get("elapsed") or "",
            "description": it.get("description") or it.get("event") or it.get("text") or "",
            "predicted_tags": ",".join((it.get("predicted_tags") or [])) if isinstance(it.get("predicted_tags"), list) else ""
        })
as_df(rows)



## 10) (Optional) Highlight videos
If available from the provider, fetch `video.highlights` for the picked event.


In [ ]:

vids = router.handle({"intent": "video.highlights", "args": {"eventId": str(event_id)}})
print("OK:", vids.get("ok"))
data = vids.get("data") or {}
items = data.get("result") or data.get("event") or data.get("videos") or []
items = items if isinstance(items, list) else [items]

show = []
for v in items[:5]:
    show.append({
        "title": v.get("video_title") or v.get("title") or v.get("strTitle"),
        "url": v.get("video_url") or v.get("url") or v.get("strVideo"),
        "source": v.get("video_source") or v.get("source") or v.get("strCreator")
    })
as_df(show)



## Appendix — Picking a different match
- To select a different live match, change the index `live_list[0]` to another item (e.g., `live_list[3]`).
- Or filter by team or league before picking:
```python
targets = [ev for ev in live_list if (ev.get("league_name") == "Premier League")]
chosen = targets[0]
```
Make sure to re-run the downstream cells after changing `chosen`.
